In [1]:
import lightgbm as lgb
import pandas as pd
import catboost as cb
import xgboost as xgb

Load data
====

In [2]:
epsilon_test = pd.read_csv('test', header=None, sep='\t', nrows=100000)

In [3]:
X_test = epsilon_test.drop(0, axis=1).values
xgb_test = xgb.DMatrix(X_test)
catboost_pool = cb.Pool(X_test)

In [4]:
epsilon_model = cb.CatBoost()
epsilon_model.load_model('epsilon8k_64.bin')
print epsilon_model.tree_count_

8000


32 thread apply
====

In [5]:
bst = xgb.Booster({'nthread': 32})  # set 32 thread in openmp threadpool
bst.load_model('XGBoost.model')  # load data
lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [6]:
%timeit -r 5 bst.predict(xgb_test, ntree_limit=8000)

1 loop, best of 5: 4.8 s per loop


In [7]:
%timeit -r 5 lgb_model.predict(X_test, num_iteration=8000)

The slowest run took 5.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 18 s per loop


In [8]:
%timeit -r 5 epsilon_model.predict(catboost_pool, thread_count=32, ntree_end=8000)

1 loop, best of 5: 145 ms per loop


1 thread apply
====

In [9]:
bst = xgb.Booster({'nthread': 1})  # set 1 thread in openmp threadpool
bst.load_model('XGBoost.model')  # load data
lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [13]:
%time _ = bst.predict(xgb_test, ntree_limit=8000)

CPU times: user 1min 11s, sys: 316 ms, total: 1min 11s
Wall time: 1min 11s


In [14]:
%time _ = lgb_model.predict(X_test, num_iteration=8000)

CPU times: user 1min 22s, sys: 5.78 s, total: 1min 28s
Wall time: 1min 28s


In [15]:
%time _ = epsilon_model.predict(catboost_pool, thread_count=1, ntree_end=8000)

CPU times: user 1.84 s, sys: 0 ns, total: 1.84 s
Wall time: 1.83 s
